<a href="https://colab.research.google.com/github/MedicalImageAnalysisTutorials/DeepLearning4All/blob/main/IA_DNN_ImageRegistration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  TODO 13.10.2021

#  pix2pix check, publish  and run local  (still to do) for Jing  
#  classification revisited  (everyone)
#  image registration: 
    # organize and add theory 
    # spatial transformers 


#  image registration: 
#  spatial transformers: https://colab.research.google.com/github/sayakpaul/Spatial-Transformer-Networks-with-Keras/blob/main/STN.ipynb
#  voxelmorph: https://github.com/voxelmorph/voxelmorph
#  ICNet: https://github.com/zhangjun001/ICNet
#  labIRN: https://github.com/cwmok/LapIRN
#  LWM https://github.com/idhamari/learning_without_metric?organization=idhamari&organization=idhamari 

# Introduction

https://github.com/MedicalImageAnalysisTutorials/ImageRegistrationTutorial/blob/master/m2p_image_registration_example_mse_translation_GradientDescent.ipynb

In this notebook, I will try to provide a practical tutorial for deep learning using simple examples. I will try to use simple implementation and avoid using built-in functions to give clear idea about the concept. You need basic programming knowledge. 

**The Image Registration  Problem:**


An example:

     






In [2]:
# Setup 
doInstall =1
if doInstall:
  !pip install SimpleITK

import os, random, time, math, io
import numpy as np
import matplotlib.pyplot as plt
import cv2 

try:   
  import SimpleITK as sitk 
except:
  print("please install simpleitk") 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models



# to reproduce the same results given same input
np.random.seed(1)               


# Reading and exploring the datasets



In [3]:
datasetID = 1  # 1:minst is selected by default, for cifar10 use 2
NNID      = 1  # 1:NN is by default, for DNN use 2,or 3, for 3D use 4  
number_of_classes = 10  # each datasets have 10 classes
showSamples = 1
# if you have large GPU memory you can combine the images to batches 
# for faster training.
# It is good to try different values
batch_size = 2 # you can try larger batch size e.g. 1024 * 6


IMG_SIZE = 28  # for minst 28
IMG_CH   = 1   # for minst 1

BATCH_SIZE = 128

doAug = 0

# # minst dataset
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# class_names = range(10)
# if datasetID==2:
#     # cifar10 dataset
#     # The CIFAR10 dataset contains 60,000 color images in 10 classes, 
#     # with 6,000 images in each class.
#     # The dataset is divided into 50,000 training images and 10,000 testing images.
#     # The classes are mutually exclusive and there is no overlap between them.

#     (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
#     class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']
#     if NNID==4:
#         #TODO: fix this 
#         showSamples =0
        
#         x_train = x_train.reshape(-1,32*32*3) # (32 * 32 * 3)        
#         x_train = np.resize(x_train,(x_train.shape[0],15,15,15))        
#         x_train = x_train.reshape(-1,15,15,15)
#         x_test = x_test.reshape(-1,32*32*3) # (32 * 32 * 3)
#         x_test = np.resize(x_test,(x_test.shape[0],15,15,15))
#         x_test = x_test.reshape(-1,15,15,15)
#         x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
#         y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
#         x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
#         y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


#         print(x_train.shape)
#         print(x_test.shape)


# # get size 
# h = x_train.shape[1] # image height
# w = x_train.shape[2] # image width
# # check for rgb 
# try:
#     # number of channels
#     c =  x_train.shape[3]
# except:
#     # number of channels
#     c =  1
#     # if there is no number of channels, add 1
#     x_train  =  x_train[..., np.newaxis] # np.reshape(x_train, (-1, h,w,1))
#     y_train  =  y_train[..., np.newaxis] # np.reshape(y_train, (-1, h,w,1))
#     x_test   =  x_test[..., np.newaxis]  # np.reshape(x_test,  (-1, h,w,1))
#     y_test   =  y_test[..., np.newaxis]  # np.reshape(y_test,  (-1, h,w,1))


# # Reserve 10,000 samples for validation.
# x_val = x_train[-10000:]
# y_val = y_train[-10000:]
# x_train = x_train[:-10000]
# y_train = y_train[:-10000]

# number_of_pixels = h * w * c


# print("dataset shape   : ",x_train.shape)
# print("number of images: ",x_train.shape[0])
# print("image size      : ",x_train[0].shape)
# print("image data type : ",type(x_train[0][0][0][0]))
# print("image max  value: ",np.max(x_train[0]))
# print("image min  value: ",np.min(x_train[0]))
# if c==1:
#    print("gray or binary image (not color image)")
# elif c==3:
#    print("rgb color image (or probably non-color image represented with 3 channels)")


# # display sample images 
# if showSamples:
#     plt.figure(figsize=(10,10))
#     for i in range(25):
#         plt.subplot(5,5,i+1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.grid(False)
#         #plt.imshow(x_train[i])
#         plt.imshow(cv2.cvtColor(x_train[i], cv2.COLOR_BGR2RGB))

#         # The CIFAR labels happen to be arrays, 
#         # which is why you need the extra index
#         if datasetID==1:
#             plt.xlabel(y_train[i])
#         elif datasetID==2:
#             plt.xlabel(class_names[y_train[i][0]])
#     plt.show()


# # normalisation
# x_train = np.array([ x/255.0 for x in x_train])
# x_val   = np.array([ x/255.0 for x in x_val])
# x_test  = np.array([ x/255.0 for x in x_test])
# #y_train = y_train.astype(np.float32)

# # for NN we need 1D 
# if NNID ==1:
#    x_train = np.reshape(x_train, (-1, number_of_pixels))
#    x_val   = np.reshape(x_val,  (-1, number_of_pixels))
#    x_test  = np.reshape(x_test , (-1, number_of_pixels))

# # Prepare the training dataset.
# print(x_train.shape,y_train.shape)
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
# #train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# # Prepare the validation dataset.
# val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
# val_dataset = val_dataset.batch(batch_size)

# # Prepare the test dataset.
# tst_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# tst_dataset = tst_dataset.batch(batch_size)


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, IMG_SIZE, IMG_SIZE, IMG_CH))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, IMG_SIZE, IMG_SIZE, IMG_CH))

# Put aside a few samples to create our validation set
val_samples = 2000
x_val, y_val = x_train[:val_samples], y_train[:val_samples]
new_x_train, new_y_train = x_train[val_samples:], y_train[val_samples:]

train_ds = (
    tf.data.Dataset.from_tensor_slices((new_x_train, new_y_train))
    .shuffle(BATCH_SIZE * 100)
    .batch(BATCH_SIZE)
)

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)


# Dataset augmentation

It is important to train the model on different variations of the dataset. It is also important to have large datset for training.

Using dataset augmentation helps to achieve both of the above goals. From one image, one can generate hundred thousands of images using image transformation.

The image transformation could be [spatial transform]() or point transform where we move the points of the image to new locations e.g. shifting, flipping, and/or rotating the imag. 

Another type of transformation is intensity transform or pixel transform where we change the color values of the pixels in the image e.g. invert the color, add more brightness or darkness. 



In [4]:
def imagePixelTransforms(img):    
    images = []
    # let's make 3 simple transformations
    img1   = 1.0- img # invert color
    img2   = img +0.3 # more brightness
    img3   = img -0.3 # more darkness
    images = np.array([img1,img2,img3])
    images = [ img.reshape(img.shape) for img in images]

    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    return images

def imagePointTransforms(img):
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    center  = (img.shape[0] / 2, img.shape[1] / 2)
    sz      = (img.shape[1], img.shape[0])
    tMatrix = cv2.getRotationMatrix2D(center, 45, 1)
    img1 = cv2.warpAffine(img, tMatrix, sz)
    img1 = img1[...,np.newaxis] if img1.shape !=img.shape else img1
    tMatrix = cv2.getRotationMatrix2D(center, 90, 1)
    img2 = cv2.warpAffine(img, tMatrix, sz)
    img2 = img2[...,np.newaxis] if img2.shape !=img.shape else img2
    tMatrix = cv2.getRotationMatrix2D(center, 270, 1)
    img3 = cv2.warpAffine(img, tMatrix, sz)
    img3 = img3[...,np.newaxis] if img3.shape !=img.shape else img3

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images


# define a function for sitk transform
def resample(img_array, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    image = sitk.GetImageFromArray(img_array)
    reference_image = image
    interpolator = sitk.sitkCosineWindowedSinc
    default_value = 100.0
    resampled_img = sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)
    resampled_array = sitk.GetArrayFromImage(resampled_img)
    return resampled_array

def affine_rotate(transform, degrees):
    parameters = np.array(transform.GetParameters())
    new_transform = sitk.AffineTransform(transform)
    dimension =3 
    matrix = np.array(transform.GetMatrix()).reshape((dimension,dimension))
    radians = -np.pi * degrees / 180.
    rotation = np.array([[1  ,0,0], 
                         [0, np.cos(radians), -np.sin(radians)],
                         [0, np.sin(radians), np.cos(radians)]]
                        )
    new_matrix = np.dot(rotation, matrix)
    new_transform.SetMatrix(new_matrix.ravel())
    return new_transform


def imagePoint3DTransforms(img):
    #print("imagePoint3DTransforms")
    images = []
    # let's make 3 simple transformations
    # Perform the rotation
    # In SimpleITK resampling convention, the transformation maps points 
    # from the fixed image to the moving image,
    # so inverse of the transform is applied

    center = (img.shape[0] /2, img.shape[1] /2,img.shape[1] /2)
    rotation_around_center = sitk.AffineTransform(3)
    rotation_around_center.SetCenter(center)
    
    rotation_around_center = affine_rotate(rotation_around_center, -45)
    img1 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img2 = resample(img, rotation_around_center)

    rotation_around_center = affine_rotate(rotation_around_center, -90)
    img3 = resample(img, rotation_around_center)

    images = np.array([img1,img2,img3])
    # plt.figure() ;    plt.imshow(img)
    # plt.figure() ;    plt.imshow(img1)
    # plt.figure() ;    plt.imshow(img2)
    # plt.figure() ;    plt.imshow(img3)    
    #plt.figure() ;    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.figure() ;    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    return images

def doAugmentation(images,labels,batch_size):
    # input is an image or a batch e.g. list of images 
    # get numpy arrays from the tensor    
    images = images.numpy()
    labels = labels.numpy()
    # if 1d convert back to 2d
    #print(images.shape)
    rgb = 0 ; is3d = 0
    if len(images.shape) == 2:
       try: 
          img2d_shape = int(math.sqrt(images.shape[1])) # gray or binary image
          images =images.reshape(-1,img2d_shape,img2d_shape)
       except:
          try: 
            img2d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            images =images.reshape(-1,img2d_shape,img2d_shape,3)
            rgb = 1  
          except:
            pass  
            # img3d_shape = int(math.sqrt(images.shape[1]/3)) # rgb image
            # images =images.reshape(-1,img2d_shape,img2d_shape,3)
            # is3d = 1  



    x_outputs = [] ; y_outputs = []
    i = 0
    for img in images:
        #print("-------------------------", i ,"--------------------")
        if NNID==4:
           img = img.squeeze() 
        # from each images we generate 6 images
        # 64 batch will generate 448
        x_outputs.extend([img])
        imgs1 = imagePoint3DTransforms(img)
        imgs2 = imagePixelTransforms(img)
        #if not rgb:
           #imgs1 = np.array( x[...,np.newaxis] for x in imgs1 if len(x.shape)<3) 
           #imgs2 = np.array( x[...,np.newaxis] for x in imgs2 if len(x.shape)<3)
        x_outputs.extend(imgs1) # 3 images
        x_outputs.extend(imgs2) # 3 images
        # print(img.shape)
        # print(imgs1[0].shape)
        # print(imgs2[0].shape)
        # assign the same label to all transformed images
        for j in range ( len(imgs1) +len(imgs2)+1):
            y_outputs.extend([labels[i]])

        i = i +1
    x_outputs = np.array(x_outputs)
    if NNID==4:
       x_outputs = np.array([x[...,np.newaxis] for x in x_outputs])
    y_outputs = np.array(y_outputs)

    if (not rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape,1))
    elif (rgb) and (NNID==1):
       x_outputs = np.reshape(x_outputs, (-1,img2d_shape*img2d_shape*3))   

    new_train_dataset = tf.data.Dataset.from_tensor_slices((x_outputs, y_outputs))
    new_train_dataset = new_train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    return new_train_dataset

# Spatial Transform Network (STN)

https://colab.research.google.com/github/sayakpaul/Spatial-Transformer-Networks-with-Keras/blob/main/STN.ipynb


https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/spatial_transformer_tutorial.ipynb




## HyperParameters


In [5]:
AUTO = tf.data.AUTOTUNE
EPOCHS = 30

## Creating the model

In [6]:
# model creation



#----------------------------------------------------
# Grid Generator
# generate a deformation field using transformation matrix theta
#----------------------------------------------------
def affine_grid_generator(height, width, theta):
    """
    This function returns a sampling grid, which when
    used with the bilinear sampler on the input feature
    map, will create an output feature map that is an
    affine transformation [1] of the input feature map.
    Input
    -----
    - height: desired height of grid/output. Used
      to downsample or upsample.
    - width: desired width of grid/output. Used
      to downsample or upsample.
    - theta: affine transform matrices of shape (num_batch, 2, 3).
      For each image in the batch, we have 6 theta parameters of
      the form (2x3) that define the affine transformation T.
    Returns
    -------
    - normalized grid (-1, 1) of shape (num_batch, 2, H, W).
      The 2nd dimension has 2 components: (x, y) which are the
      sampling points of the original image for each point in the
      target image.
    Note
    ----
    [1]: the affine transformation allows cropping, translation,
         and isotropic scaling.
    """
    num_batch = tf.shape(theta)[0]

    # create normalized 2D grid
    x = tf.linspace(-1.0, 1.0, width)
    y = tf.linspace(-1.0, 1.0, height)
    x_t, y_t = tf.meshgrid(x, y)

    # flatten
    x_t_flat = tf.reshape(x_t, [-1])
    y_t_flat = tf.reshape(y_t, [-1])

    # reshape to [x_t, y_t , 1] - (homogeneous form)
    ones = tf.ones_like(x_t_flat)
    sampling_grid = tf.stack([x_t_flat, y_t_flat, ones])

    # repeat grid num_batch times
    sampling_grid = tf.expand_dims(sampling_grid, axis=0)
    sampling_grid = tf.tile(sampling_grid, tf.stack([num_batch, 1, 1]))

    # cast to float32 (required for matmul)
    theta = tf.cast(theta, 'float32')
    sampling_grid = tf.cast(sampling_grid, 'float32')

    # transform the sampling grid - batch multiply
    batch_grids = tf.matmul(theta, sampling_grid)
    # batch grid has shape (num_batch, 2, H*W)

    # reshape to (num_batch, H, W, 2)
    batch_grids = tf.reshape(batch_grids, [num_batch, 2, height, width])

    return batch_grids    

#----------------------------------------------------
# BiLinear Sampler
# transform an image using deformation field 
#----------------------------------------------------
def bilinear_sampler(img, defField):
    """
    Performs bilinear sampling of the input images according to the
    normalized coordinates provided by the sampling grid. Note that
    the sampling is done identically for each channel of the input.
    To test if the function works properly, output image should be
    identical to input image when theta is initialized to identity
    transform.
    Input
    -----
    - img: batch of images in (B, H, W, C) layout.
    - grid: x, y which is the output of affine_grid_generator.
    Returns
    -------
    - out: interpolated images according to grids. Same size as grid.
    """

    def get_pixel_value(img, x, y):
        """
        Utility function to get pixel value for coordinate
        vectors x and y from a  4D tensor image.
        Input
        -----
        - img: tensor of shape (B, H, W, C)
        - x: flattened tensor of shape (B*H*W,)
        - y: flattened tensor of shape (B*H*W,)
        Returns
        -------
        - output: tensor of shape (B, H, W, C)
        """
        shape = tf.shape(x)
        batch_size = shape[0]
        height = shape[1]
        width = shape[2]

        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, height, width))

        indices = tf.stack([b, y, x], 3)

        return tf.gather_nd(img, indices)

    x =  defField[:, 0, :, :]
    y =  defField[:, 1, :, :]
    H = tf.shape(img)[1]
    W = tf.shape(img)[2]
    max_y = tf.cast(H - 1, 'int32')
    max_x = tf.cast(W - 1, 'int32')
    zero = tf.zeros([], dtype='int32')

    # rescale x and y to [0, W-1/H-1]
    x = tf.cast(x, 'float32')
    y = tf.cast(y, 'float32')
    x = 0.5 * ((x + 1.0) * tf.cast(max_x-1, 'float32'))
    y = 0.5 * ((y + 1.0) * tf.cast(max_y-1, 'float32'))

    # grab 4 nearest corner points for each (x_i, y_i)
    x0 = tf.cast(tf.floor(x), 'int32')
    x1 = x0 + 1
    y0 = tf.cast(tf.floor(y), 'int32')
    y1 = y0 + 1

    # clip to range [0, H-1/W-1] to not violate img boundaries
    x0 = tf.clip_by_value(x0, zero, max_x)
    x1 = tf.clip_by_value(x1, zero, max_x)
    y0 = tf.clip_by_value(y0, zero, max_y)
    y1 = tf.clip_by_value(y1, zero, max_y)

    # get pixel value at corner coords
    Ia = get_pixel_value(img, x0, y0)
    Ib = get_pixel_value(img, x0, y1)
    Ic = get_pixel_value(img, x1, y0)
    Id = get_pixel_value(img, x1, y1)

    # recast as float for delta calculation
    x0 = tf.cast(x0, 'float32')
    x1 = tf.cast(x1, 'float32')
    y0 = tf.cast(y0, 'float32')
    y1 = tf.cast(y1, 'float32')

    # calculate deltas
    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    # add dimension for addition
    wa = tf.expand_dims(wa, axis=3)
    wb = tf.expand_dims(wb, axis=3)
    wc = tf.expand_dims(wc, axis=3)
    wd = tf.expand_dims(wd, axis=3)

    # compute output
    out = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])

    return out   

#----------------------------------------------------
# Spatial transformer network forward function
#----------------------------------------------------
def stn(inputs):
    # we can test using stn or not 
    useSTN = 1
    if useSTN:
        #----------------------------------------------------
        # Spatial transformer localization-network
        #  2 conv layers followed by maxpooling
        #  the output is the transformation parameters?
        #----------------------------------------------------
        x10 =  tf.keras.Sequential([
            layers.Conv2D(8, kernel_size=7, input_shape=(IMG_SIZE, IMG_SIZE, IMG_CH), 
                        activation="relu", kernel_initializer="he_normal"),
            layers.MaxPool2D(strides=2),

            layers.Conv2D(10, kernel_size=5, 
                        activation="relu", kernel_initializer="he_normal"),
            layers.MaxPool2D(strides=2),
        ])(inputs)

        #----------------------------------------------------
        # Regressor for the 3 * 2 affine matrix
        #  we initialize with identity matrix to get a similar image
        #----------------------------------------------------       
        x20 = tf.reshape(x10, (-1, 10 * 3 * 3 ))
        
        tMatrix =  tf.keras.Sequential([
            layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
            layers.Dense(3 * 2, kernel_initializer="zeros", 
                         bias_initializer=tf.keras.initializers.Constant([1, 0, 0, 0, 1, 0]))
        ])(x20)
        # reshape to output an affine transformation matrix 
        tMatrix = tf.reshape(tMatrix, (-1, 2, 3))
        # we can output the transfrom parameters here 

        # deformation field? using image size and the transformation matrix
        grid = affine_grid_generator(IMG_SIZE, IMG_SIZE, tMatrix)
        # x_s = grid[:, 0, :, :]
        # y_s = grid[:, 1, :, :]
        # transform the image using the deformation field        
        new_image = bilinear_sampler(inputs, grid)
        # we can output the transformed image here 
    else:
        new_image = inputs

    # this is just a classification network
    x41 = layers.Conv2D(10, (5, 5), activation="relu", kernel_initializer="he_normal")(new_image)
    x42 = layers.MaxPooling2D(pool_size=(2, 2))(x41)
    
    x51 = layers.Conv2D(20, (5, 5), activation="relu", kernel_initializer="he_normal")(x42)
    x52 = layers.SpatialDropout2D(0.5)(x51)
    x53 = layers.MaxPooling2D(pool_size=(2, 2))(x52)
    
    x61 = tf.reshape(x53, (-1, 320))
    x62 = layers.Dense(50, activation="relu", kernel_initializer="he_normal")(x61)
    x63 = layers.Dropout(0.5)(x62)    
    outputs = layers.Dense(10, activation="softmax")(x63)
    model = tf.keras.Model(inputs, outputs)    
    return model

  

## Training


In [7]:
# num_params_w_stn = get_classification_model().count_params()
# print(f"Number of parameters    with STN: {num_params_w_stn / 1e6} M")

# transformed_x = stn(np.expand_dims(x_train[0], 0))
# plt.imshow(transformed_x.numpy().squeeze(), cmap="binary")
# plt.show()
# plt.imshow(x_train[0].squeeze(), cmap="binary")
# plt.show()



inputs = layers.Input((IMG_SIZE, IMG_SIZE, IMG_CH))    
stn_model = stn(inputs)

# stn_mini_model = tf.keras.Model(stn_model.input, stn_model.layers[-10].output)
# file_writer_cm = tf.summary.create_file_writer("logs/stn_progress")

# # Courtesy: https://www.tensorflow.org/tensorboard/image_summaries
# def plot_to_image(figure):
#     # Save the plot to a PNG in memory.
#     buf = io.BytesIO()
#     plt.savefig(buf, format='png')
    
#     # Closing the figure prevents it from being displayed directly inside
#     # the notebook.
#     plt.close(figure)
#     buf.seek(0)
    
#     # Convert PNG buffer to TF image
#     image = tf.image.decode_png(buf.getvalue(), channels=4)
    
#     # Add the batch dimension
#     image = tf.expand_dims(image, 0)
#     return image

# def log_progression(epoch, logs):
#     sampled_transformed = stn_mini_model(x_test[:25])
    
#     figure = plt.figure(figsize=(10, 10))
#     for n in range(25):
#         plt.subplot(5, 5, n+1)
#         plt.imshow(sampled_transformed[n].numpy().squeeze(), cmap="binary")
#         plt.axis("off")

#     progress_image = plot_to_image(figure)
#     with file_writer_cm.as_default():
#         tf.summary.image("Progression", progress_image, step=epoch)

# # Define the per-epoch callback.
# progress_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_progression)

# es_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau()

stn_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# stn_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[es_callback, reduce_lr, progress_callback])%
stn_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
_, test_acc = stn_model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

Epoch 1/30
454/454 [==============================] - ETA: 0s - loss: 0.8085 - accuracy: 0.7349

KeyboardInterrupt: ignored

In [ ]:
# TODO
# NN pytorch


In [ ]:
# TODO
# Pytorch


In [ ]:
##TODO pytorch

In [ ]:
# TODO Pytorch


# VoxelMorph

# ICNet

#  
Training large datsets takes hours. It is good to check the [training curves](https://en.wikipedia.org/wiki/Learning_curve) and stop the training if it is a waste of time and resources. To do this one should check the training curves and identify specific problems.


In [ ]:
#TODO: show examples using the above datasets
# https://machinelearningmastery.com/diagnose-overfitting-underfitting-lstm-models/

# LabIRN



# LWM



# More resources:

* 3Blue1Brown Neural Network [video tutorials](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) 
* Deep Learning Video Lectures by Prof. Andreas Maier [Winter 20/21](https://www.youtube.com/watch?v=SCFToE1vM2U&list=PLpOGQvPCDQzvJEPFUQ3mJz72GJ95jyZTh)
* Some of the code in this notebook is taken from [here](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
* Calculating number of parameters in [CNN](https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d)
* Some of the code in this notebook is taken from [here](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb)
* https://imerit.net/blog/top-13-machine-learning-image-classification-datasets-all-pbm/
* https://nihcc.app.box.com/v/ChestXray-NIHCC
* https://www.kaggle.com/xhlulu/recursion-cellular-image-classification-224-jpg
* https://www.tensorflow.org/datasets/catalog/patch_camelyon
